In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
day_data = pd.read_csv("data/day.csv")
hour_data = pd.read_csv("data/hour.csv")

In [3]:
dd = day_data.copy()
hd = hour_data.copy()

In [4]:
dd.drop(['season'], axis=1, inplace=True)
hd.drop(['season'], axis=1, inplace=True)
dd.drop('instant', axis=1, inplace=True)
hd.drop('instant', axis=1, inplace=True)

In [5]:
# Renaming the columns

dd.rename(columns={'dteday': 'date', 'cnt': 'count'}, inplace=True)
hd.rename(columns={'dteday': 'date', 'hr': 'hour',
          'cnt': 'count'}, inplace=True)

In [6]:
# Convertin strings to pandas datetime object

dd["date"] = pd.to_datetime(dd["date"])
hd["date"] = pd.to_datetime(hd["date"])

In [7]:
def categorize_time(hour):
    if 0 <= hour < 6:
        return 0
    elif 6 <= hour < 12:
        return 1
    elif 12 <= hour < 18:
        return 2
    else:
        return 3

hd['tod'] = hd['hour'].apply(categorize_time)

In [8]:
# Getting the correct season code from datetime

def categorize_season(date):
    month = date.month
    if 2 < month < 6:
        return 0
    if 5 < month < 9:
        return 1
    if 8 < month < 12:
        return 2
    else:
        return 3

dd['season'] = dd['date'].apply(categorize_season)
hd['season'] = hd['date'].apply(categorize_season)

In [9]:
hd['datetime'] = hd['date'] + hd['hour'].astype('timedelta64[h]')
hd.drop('date', axis=1, inplace=True)

In [10]:
# Getting the humidity data from Historical Weather API
# Making a request 

import requests
response = requests.get(url="https://archive-api.open-meteo.com/v1/archive?latitude=38.8951&longitude=-77.0364&start_date=2011-01-01&end_date=2012-12-31&hourly=relativehumidity_2m,rain&daily=rain_sum,snowfall_sum&timezone=America%2FNew_York")
response.raise_for_status()

In [11]:
# Reading json and converting it to DataFrame

api_data = pd.read_json(response.text)
hourly_time = pd.to_datetime(pd.Series(api_data['hourly'][0]))
hourly_hum = pd.Series(api_data['hourly'][1]) 
hourly_rain = pd.Series(api_data['hourly'][2])
daily_time = pd.to_datetime(pd.Series(api_data['daily'][0]))
daily_rain = pd.Series(api_data['daily'][3])
daily_snow = pd.Series(api_data['daily'][4])
hour_frame = {
    'datetime': hourly_time,
    'humidity': hourly_hum,
    'rain': hourly_rain
}
day_frame = {
    'date': daily_time,
    'rain': daily_rain,
    'snow': daily_snow
}                         
api_hour_data = pd.DataFrame(hour_frame)
api_day_data = pd.DataFrame(day_frame)
display(api_hour_data)
display(api_day_data)    

,datetime,humidity,rain
0,2011-01-01 00:00:00,93,0.0
1,2011-01-01 01:00:00,94,0.0
2,2011-01-01 02:00:00,96,0.0
3,2011-01-01 03:00:00,96,0.0
4,2011-01-01 04:00:00,98,0.0
...,...,...,...
17539,2012-12-31 19:00:00,54,0.0
17540,2012-12-31 20:00:00,54,0.0
17541,2012-12-31 21:00:00,54,0.0
17542,2012-12-31 22:00:00,55,0.0


,date,rain,snow
0,2011-01-01,2.5,0.00
1,2011-01-02,3.0,0.00
2,2011-01-03,0.0,0.00
3,2011-01-04,0.0,0.00
4,2011-01-05,0.0,0.00
...,...,...,...
726,2012-12-27,1.6,0.00
727,2012-12-28,0.0,0.00
728,2012-12-29,2.7,2.87
729,2012-12-30,0.0,0.00


In [12]:
# Merging the original dataset with the retreived weather data
hd = pd.merge(hd, api_hour_data, how='inner', on='datetime')
dd = pd.merge(dd, api_day_data, how='inner', on='date')

In [13]:
# Replacing humidity values with new ones and leaving one humidity column

hd['hum'] = hd['humidity']
hd.drop('humidity', axis=1, inplace=True)

In [14]:
# Replacing humidity values in day data with the average of the day humidity from hour data 

dd['hum'] = np.round(hd.groupby(hd['datetime'].dt.date)['hum'].mean().values, 2)
dd.head()

,date,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,count,season,rain,snow
0,2011-01-01,0,1,0,6,0,2,0.344167,0.363625,92.33,0.160446,331,654,985,3,2.5,0.0
1,2011-01-02,0,1,0,0,0,2,0.363478,0.353739,75.09,0.248539,131,670,801,3,3.0,0.0
2,2011-01-03,0,1,0,1,1,1,0.196364,0.189405,42.82,0.248309,120,1229,1349,3,0.0,0.0
3,2011-01-04,0,1,0,2,1,1,0.200000,0.212122,58.09,0.160296,108,1454,1562,3,0.0,0.0
4,2011-01-05,0,1,0,3,1,1,0.226957,0.229270,59.74,0.186900,82,1518,1600,3,0.0,0.0


In [15]:
dd.loc[dd['snow'] > 4].sort_values(by='snow')

,date,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,count,season,rain,snow
301,2011-10-29,0,10,0,6,0,3,0.254167,0.227913,96.25,0.351371,57,570,627,2,22.3,5.18
725,2012-12-26,1,12,0,3,1,3,0.243333,0.220333,84.58,0.316546,9,432,441,3,14.3,6.02
25,2011-01-26,0,1,0,3,1,3,0.217500,0.203600,90.31,0.293850,34,472,506,3,3.6,14.84


In [16]:
# Replacing outlier value 

dd.loc[dd['rain'] > 20, 'rain'] = dd.loc[dd['rain'] < 21, 'rain'].max()
dd.loc[dd['snow'] > 4, 'snow'] = dd.loc[dd['snow'] < 5, 'snow'].max()
hd.loc[hd['rain'] > 6, 'rain'] = dd.loc[dd['rain'] < 7, 'rain'].max()

In [17]:
dd.sort_values(by='rain', ascending=False)

,date,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,count,season,rain,snow
477,2012-04-22,1,4,0,0,0,3,0.396667,0.389504,89.79,0.344546,120,907,1027,0,20.0,0.00
626,2012-09-18,1,9,0,2,1,2,0.623333,0.565067,95.38,0.357587,371,3702,4073,2,20.0,0.00
64,2011-03-06,0,3,0,0,0,2,0.376522,0.366252,90.61,0.343287,114,491,605,0,20.0,0.00
68,2011-03-10,0,3,0,4,1,3,0.389091,0.385668,95.55,0.261877,46,577,623,0,20.0,0.00
554,2012-07-08,1,7,0,0,0,1,0.822500,0.790396,61.38,0.125629,1203,3469,4672,1,20.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,2011-05-29,0,5,0,0,0,1,0.667500,0.615550,75.33,0.213938,2355,2433,4788,0,0.0,0.00
416,2012-02-21,1,2,0,2,1,1,0.287826,0.295113,70.13,0.205717,163,3614,3777,3,0.0,0.00
415,2012-02-20,1,2,1,1,0,1,0.280000,0.273391,53.48,0.229083,502,2627,3129,3,0.0,0.07
414,2012-02-19,1,2,0,0,0,2,0.280000,0.265788,54.71,0.253112,618,2071,2689,3,0.0,0.00


In [18]:
# Normalizing the data from the API

hd['rain'] = (hd['rain'] - hd['rain'].min()) / (hd['rain'].max() - hd['rain'].min())
hd['hum'] = (hd['hum'] - hd['hum'].min()) / (hd['hum'].max() - hd['hum'].min()) 
dd['snow'] = (dd['snow'] - dd['snow'].min()) / (dd['snow'].max() - dd['snow'].min()) 
dd['rain'] = (dd['rain'] - dd['rain'].min()) / (dd['rain'].max() - dd['rain'].min()) 
dd['hum'] = (dd['hum'] - dd['hum'].min()) / (dd['hum'].max() - dd['hum'].min()) 

In [19]:
# Creating a new column israin where 1 mean more than 0 rain precipitation 

hd['israin'] = (hd['rain'] > 0).astype(int)

In [20]:
dd['israin'] = (dd['rain'] > 0).astype(int)
dd['issnow'] = (dd['snow'] > 0).astype(int)

In [21]:
dd['season'] = dd['season'].astype('int')
hd['season'] = hd['season'].astype('int')


In [22]:
dd.dtypes

date          datetime64[ns]
yr                     int64
mnth                   int64
holiday                int64
weekday                int64
workingday             int64
weathersit             int64
temp                 float64
atemp                float64
hum                  float64
windspeed            float64
casual                 int64
registered             int64
count                  int64
season                 int32
rain                 float64
snow                 float64
israin                 int32
issnow                 int32
dtype: object

In [23]:
hd.dtypes

yr                     int64
mnth                   int64
hour                   int64
holiday                int64
weekday                int64
workingday             int64
weathersit             int64
temp                 float64
atemp                float64
hum                  float64
windspeed            float64
casual                 int64
registered             int64
count                  int64
tod                    int64
season                 int32
datetime      datetime64[ns]
rain                 float64
israin                 int32
dtype: object

In [24]:
dd.describe()

,date,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,count,season,rain,snow,israin,issnow
count,731,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,2012-01-01 00:00:00,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.590669,0.190486,848.176471,3656.172367,4504.348837,1.492476,0.117244,0.015475,0.465116,0.050616
min,2011-01-01 00:00:00,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2011-07-02 12:00:00,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.465449,0.134950,315.500000,2497.000000,3152.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2012-01-01 00:00:00,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.591817,0.180975,713.000000,3662.000000,4548.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,2012-07-01 12:00:00,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730706,0.233214,1096.000000,4776.500000,5956.000000,2.000000,0.105000,0.000000,1.000000,0.000000
max,2012-12-31 00:00:00,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,1.000000,0.507463,3410.000000,6946.000000,8714.000000,3.000000,1.000000,1.000000,1.000000,1.000000
std,NaN,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.188463,0.077498,686.622488,1560.256377,1937.211452,1.118162,0.239732,0.098044,0.499123,0.219362


In [25]:
hd.to_csv('data/hd.csv', index=False)
dd.to_csv('data/dd.csv', index=False)